In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

In [2]:
df = pd.read_csv("Train.csv")
df.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


In [4]:
df.agg({
    "Cost_of_the_Product": ["count", "mean", "min", "max"],
    "Discount_offered": ["mean", "min", "max"],
    "Weight_in_gms": ["mean", "min", "max"]
})

,Cost_of_the_Product,Discount_offered,Weight_in_gms
count,10999.000000,NaN,NaN
mean,210.196836,13.373216,3634.016729
min,96.000000,1.000000,1001.000000
max,310.000000,65.000000,7846.000000


In [5]:
pivot1 = pd.pivot_table(
    df,
    index="Warehouse_block",
    columns="Mode_of_Shipment",
    values=["Cost_of_the_Product", "Discount_offered"],
    aggfunc=["mean", "count"]
)

pivot1

mean                                           \
                 Cost_of_the_Product                         Discount_offered   
Mode_of_Shipment              Flight        Road        Ship           Flight   
Warehouse_block                                                                 
A                         210.037037  207.268707  208.818841        13.050505   
B                         210.945946  213.312925  212.175382        12.584459   
C                         210.532203  210.938776  211.407556        12.725424   
D                         210.703704  209.116438  211.376707        13.208754   
F                         206.809122  211.105802  209.139068        13.697635   

                                                     count             \
                                       Cost_of_the_Product              
Mode_of_Shipment       Road       Ship              Flight Road  Ship   
Warehouse_block                                                         
A                 13.163265  13.277778                 297  294  1242   
B                 13.037415  13.366050                 296  294  1243   
C                 13.748299  13.477492                 295  294  1244   
D                 12.496575  13.206426                 297  292  1245   
F                 13.027304  13.811897                 592  586  2488   

                                              
                 Discount_offered             
Mode_of_Shipment           Flight Road  Ship  
Warehouse_block                               
A                             297  294  1242  
B                             296  294  1243  
C                             295  294  1244  
D                             297  292  1245  
F                             592  586  2488

In [6]:
slice_A = df[df["Warehouse_block"] == "A"]
slice_A.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
8,9,A,Flight,3,4,150,3,low,F,11,1861,1
14,15,A,Flight,4,3,152,3,low,M,43,1009,1
20,21,A,Ship,3,3,161,2,medium,F,38,1521,1
26,27,A,Ship,4,1,172,3,high,F,24,1066,1


In [7]:
dice_data = df[
    (df["Warehouse_block"].isin(["A","B"])) &
    (df["Mode_of_Shipment"] == "Flight")
]
dice_data.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
8,9,A,Flight,3,4,150,3,low,F,11,1861,1
9,10,B,Flight,3,2,164,3,medium,F,29,1187,1
14,15,A,Flight,4,3,152,3,low,M,43,1009,1


In [8]:
drill = pd.pivot_table(
    df,
    index=["Mode_of_Shipment", "Product_importance"],
    values="Cost_of_the_Product",
    aggfunc=["mean", "count"]
).sort_values(("mean","Cost_of_the_Product"), ascending=False)

drill

mean               count
                                    Cost_of_the_Product Cost_of_the_Product
Mode_of_Shipment Product_importance                                        
Ship             low                         212.725153                3602
Road             medium                      212.112752                 745
Flight           low                         210.875895                 838
Road             low                         210.421237                 857
Flight           medium                      209.028351                 776
Ship             medium                      208.838231                3233
                 high                        204.417863                 627
Road             high                        203.050633                 158
Flight           high                        202.564417                 163

In [9]:
rollup = df.groupby("Mode_of_Shipment")["Cost_of_the_Product"].agg(["mean","count"])
rollup

,mean,count
Mode_of_Shipment,,
Flight,209.306697,1777
Road,210.475568,1760
Ship,210.343072,7462


In [10]:
olap_target = df.groupby("Reached.on.Time_Y.N")[["Cost_of_the_Product","Discount_offered","Weight_in_gms"]].mean()
olap_target

,Cost_of_the_Product,Discount_offered,Weight_in_gms
Reached.on.Time_Y.N,,,
0,214.498647,5.545987,4168.668395
1,207.289197,18.663721,3272.640104
